In [1]:
import sys
sys.path.insert(0, '/home/bibek/projects/DEEPL')
import numpy as np
import pandas as pd
import csv
import json

In [5]:
from helpers.deep import get_sector_excerpt, process_deep_entries_data
data = process_deep_entries_data('nlp/nlp_out.csv')

In [12]:
import random
random.shuffle(data)

SAMPLE_SIZE = 100
sectors_samples = {}  # to hold samples of all sectors
samples_counts = {}  # to hold samples count of sectors
for excerpt, label in data:
    if not samples_counts.get(label):
        samples_counts[label] = 0
    if samples_counts[label] < SAMPLE_SIZE:
        sectors_samples[label] = sectors_samples.get(label, []) + [excerpt]
        samples_counts[label] += 1


In [26]:
def get_confidence_interval_discrete(percent, true, total):
    factor = 2 if percent == 95 else 1.5 # TODO: add for other confidence levels
    adjusted_true = true + 2
    adjusted_total = total + 4
    prob = adjusted_true/float(adjusted_total)
    print(prob)
    var = prob * (1-prob)
    err = var/adjusted_total
    sqrt = err**0.5
    error_margin = sqrt*factor
    return (prob-error_margin, prob+error_margin)

In [28]:
# now get the model
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'deepl.settings'
import django
django.setup()
from classifier.models import ClassifierModel

import pickle

model = ClassifierModel.objects.last()
classifier = pickle.loads(model.data)

confidences = {}
for label, entries in sectors_samples.items():
    total_classified = len(entries)
    correct_classified = 0
    for entry in entries:
        classified = classifier.classify(classifier.preprocess(entry))
        if classified == label:
            correct_classified += 1
    confidence95 = get_confidence_interval_discrete(95, correct_classified, total_classified)
    confidences[label] = confidence95
print(confidences)

0.8269230769230769
0.7403846153846154
0.6826923076923077
0.5673076923076923
0.8461538461538461
0.6538461538461539
0.5576923076923077
0.7692307692307693
0.7596153846153846
0.5288461538461539
0.2692307692307692
0.47115384615384615
{'Health': (0.7527296442427492, 0.9011165096034045), 'WASH': (0.6544026770562239, 0.8263665537130069), 'Shelter': (0.5914143044633599, 0.7739703109212556), 'Cross': (0.47014215660142905, 0.6644732280139556), 'Protection': (0.7753949068455539, 0.9169127854621384), 'Logistic': (0.5605452838389262, 0.7471470238533815), 'Nutrition': (0.4602891807418037, 0.6550954346428117), 'Food': (0.686602129918527, 0.8518594085430116), 'Education': (0.6758116183903501, 0.8434191508404191), 'Agriculture': (0.4309514106447313, 0.6267408970475764), 'NFI': (0.1822414767574606, 0.35622006170407783), 'Livelihood': (0.3732591029524236, 0.5690485893552687)}
